This iPyNotebook will run the dilutions and the IPTG induction of cells for the tRNA depletion experiment.

In [1]:
import opentrons.simulate
import os
from rich import print
import random

protocol_file = 'complete_protocol.py'
pre_data = 'day_3_pre.csv'
iptg_volume = 6 # In microliters. 6ul of 0.00625M IPTG is 0.01875mM IPTG in 200ul total

First, we'll define the locations of each cell on the plate. 

In [2]:
cell_locations = {
    'B2': ['GFP10', 'RBS0.125'],
    'C2': ['GFP25', 'RBS0.125'],
    'D2': ['GFP50', 'RBS0.125'],
    'E2': ['GFP75', 'RBS0.125'],

    'B3': ['GFP10', 'RBS0.25'],
    'C3': ['GFP25', 'RBS0.25'],
    'D3': ['GFP50', 'RBS0.25'],
    'E3': ['GFP75', 'RBS0.25'],

    'B4': ['GFP10', 'RBS0.5'],
    'C4': ['GFP25', 'RBS0.5'],
    'D4': ['GFP50', 'RBS0.5'],
    'E4': ['GFP75', 'RBS0.5'],

    'B5': ['GFP10', 'RBS1'],
    'C5': ['GFP25', 'RBS1'],
    'D5': ['GFP50', 'RBS1'],
    'E5': ['GFP75', 'RBS1'],

    'B6': ['GFP10', 'RBS2'],
    'C6': ['GFP25', 'RBS2'],
    'D6': ['GFP50', 'RBS2'],
    'E6': ['GFP75', 'RBS2'],

    'B7': ['GFP10', 'RBS4'],
    'C7': ['GFP25', 'RBS4'],
    'D7': ['GFP50', 'RBS4'],
    'E7': ['GFP75', 'RBS4'],

    'B8': ['blank', 'blank'],
    'C8': ['blank', 'blank'],
    'D8': ['blank', 'blank'],
    'E8': ['blank', 'blank'],
}

Now, we'll assign each preconditioning well to two experimental wells.

In [3]:
import random
random.seed('day 3')

valid_rows = list("BCDEFG")
valid_columns = ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11']

valid_wells = [f"{row}{column}" for row in valid_rows for column in valid_columns]

wells_with_cells = [well for well in cell_locations if cell_locations[well] != ['blank', 'blank']]

final_positions = {}

for well in wells_with_cells:
    position_1 = random.choice(valid_wells)
    position_1_index = valid_wells.index(position_1)
    del(valid_wells[position_1_index])
    position_2 = random.choice(valid_wells)
    position_2_index = valid_wells.index(position_2)
    del(valid_wells[position_2_index])

    final_positions[position_1] = [well, cell_locations[well][0], cell_locations[well][1]]
    final_positions[position_2] = [well, cell_locations[well][0], cell_locations[well][1]]

# Fill the rest of the valid positions with blanks

for well in valid_wells:
    final_positions[well] = final_positions.get(well, ['blank', 'blank', 'blank'])


Next, we'll calculate the appropriate well dilutions for the experiment using the data from the plate reader.

In [4]:
import pandas as pd

def parse_platereader(filename):
    """Parse the output of a Teccan plate reader and make the data tidy"""
    with open(filename, 'r') as file:
        lines = file.readlines()
    lines = [line.strip() for line in lines]
    dataframe = pd.DataFrame(columns=['time', 'well', 'label', 'value', 'temperature'])
    datasets = []
    reading_dataset = [None, None]
    start_time = None
    for i, line in enumerate(lines):
        line = line.split(',')
        if line[0].startswith('End Time:'):
            continue
        elif line[0].startswith('Start Time:'):
            start_time = i
        elif start_time and any(line) > 0 and not any(reading_dataset):
            reading_dataset[0] = i
        elif start_time and any(line) == 0 and any(reading_dataset):
            reading_dataset[1] = i # Exclusive
            datasets.append(reading_dataset)
            reading_dataset = [None, None]

    def filter_function(test_value):
        if test_value == "":
            return False
        else: 
            return True

    for dataset in datasets:
        data = lines[dataset[0]:dataset[1]]
        label = data.pop(0).split(',')[0]

        if data[0].startswith('Cycle'):
            cycle = data.pop(0).split(',')[1:]

        times = list(filter(filter_function, data.pop(0).split(',')[1:]))
        temperature = list(filter(filter_function, data.pop(0).split(',')[1:]))
        data = list(filter(filter_function, [line.split(',') for line in data]))

        for line in data:
            well = str(line.pop(0))
            line = list(filter(filter_function, line))
            for i, value in enumerate(line):
                adding_frame = pd.DataFrame({
                    'time': float(times[i]),
                    'well': well,
                    'label': label,
                    'value': float(value),
                    'temperature': float(temperature[i]),
                    }, index=[0])


                dataframe = pd.concat([dataframe, adding_frame])

    return dataframe

platereader_file = pre_data
platereader_data = parse_platereader(platereader_file)


target_od600 = 0.0234506  # This is equivilant to 0.2 OD600 on our 1cm spectrophotometer

blank_wells = [x for x in cell_locations.keys() if 'blank' in cell_locations[x]]

cell_volumes = {}
lb_volumes = {}


# Get the latest timepoint from the platereader data
latest_timepoint = platereader_data['time'].max()

# Set the OD600 column to be a float


# Get the latest OD600 values for the blank wells
blank_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'].isin(blank_wells)) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()

# Get all the wells that are used in the protocol
occupied_wells = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['label'].isin(['OD600'])), 'well']
# Remove the blank wells
occupied_wells = [well for well in occupied_wells if well not in blank_wells]

for start_well in occupied_wells:
    well_od600 = platereader_data.loc[(platereader_data['time'] == latest_timepoint) & (platereader_data['well'] == start_well) & (platereader_data['label'].isin(['OD600'])), 'value'].mean()
    well_od600 = well_od600 - blank_od600

    cell_volume = (target_od600 / well_od600) * 200
    if cell_volume > (200-iptg_volume):
        cell_volume = (200-iptg_volume)
    if cell_volume < 0:
        cell_volume = 0
    lb_volume = (200-iptg_volume) - cell_volume

    cell_volumes[start_well] = [start_well, cell_volume]
    lb_volumes[start_well] = lb_volume


FileNotFoundError: [Errno 2] No such file or directory: 'day_1_pre.csv'

Next, we associate the calculated cell volumes and LB volumes with the final well positions

In [6]:
for well in final_positions:
    source_well = final_positions[well][0]

    if source_well == 'blank':
        final_positions[well].append(0)
        final_positions[well].append((200-iptg_volume))
    else:
        final_positions[well].append(cell_volumes[source_well][1])
        final_positions[well].append(lb_volumes[source_well])

print(final_positions)

{
    'D6': ['B2', 'GFP10', 'RBS0.125', 24.802324968961695, 169.1976750310383],
    'C10': ['B2', 'GFP10', 'RBS0.125', 24.802324968961695, 169.1976750310383],
    'G8': ['C2', 'GFP25', 'RBS0.125', 22.47302474917753, 171.52697525082246],
    'D9': ['C2', 'GFP25', 'RBS0.125', 22.47302474917753, 171.52697525082246],
    'D4': ['D2', 'GFP50', 'RBS0.125', 27.45972099677744, 166.54027900322257],
    'D5': ['D2', 'GFP50', 'RBS0.125', 27.45972099677744, 166.54027900322257],
    'G9': ['E2', 'GFP75', 'RBS0.125', 30.475110891091468, 163.52488910890852],
    'D2': ['E2', 'GFP75', 'RBS0.125', 30.475110891091468, 163.52488910890852],
    'B3': ['B3', 'GFP10', 'RBS0.25', 20.6522235401309, 173.3477764598691],
    'E2': ['B3', 'GFP10', 'RBS0.25', 20.6522235401309, 173.3477764598691],
    'F5': ['C3', 'GFP25', 'RBS0.25', 21.031927614483102, 172.9680723855169],
    'C8': ['C3', 'GFP25', 'RBS0.25', 21.031927614483102, 172.9680723855169],
    'B4': ['D3', 'GFP50', 'RBS0.25', 25.37943915759807, 168.62056084240191],
    'D7': ['D3', 'GFP50', 'RBS0.25', 25.37943915759807, 168.62056084240191],
    'C3': ['E3', 'GFP75', 'RBS0.25', 30.278372877739713, 163.7216271222603],
    'F10': ['E3', 'GFP75', 'RBS0.25', 30.278372877739713, 163.7216271222603],
    'F9': ['B4', 'GFP10', 'RBS0.5', 23.42717186429252, 170.57282813570748],
    'B11': ['B4', 'GFP10', 'RBS0.5', 23.42717186429252, 170.57282813570748],
    'G11': ['C4', 'GFP25', 'RBS0.5', 20.60685456366883, 173.39314543633117],
    'B9': ['C4', 'GFP25', 'RBS0.5', 20.60685456366883, 173.39314543633117],
    'G6': ['D4', 'GFP50', 'RBS0.5', 25.50364527776358, 168.49635472223642],
    'C9': ['D4', 'GFP50', 'RBS0.5', 25.50364527776358, 168.49635472223642],
    'G4': ['E4', 'GFP75', 'RBS0.5', 19.822992513823134, 174.17700748617688],
    'F11': ['E4', 'GFP75', 'RBS0.5', 19.822992513823134, 174.17700748617688],
    'C2': ['B5', 'GFP10', 'RBS1', 24.828585152567776, 169.17141484743223],
    'C6': ['B5', 'GFP10', 'RBS1', 24.828585152567776, 169.17141484743223],
    'B7': ['C5', 'GFP25', 'RBS1', 18.249495140075457, 175.75050485992455],
    'E9': ['C5', 'GFP25', 'RBS1', 18.249495140075457, 175.75050485992455],
    'G5': ['D5', 'GFP50', 'RBS1', 24.504283161964345, 169.49571683803566],
    'F6': ['D5', 'GFP50', 'RBS1', 24.504283161964345, 169.49571683803566],
    'F2': ['E5', 'GFP75', 'RBS1', 21.753803269454018, 172.246196730546],
    'F3': ['E5', 'GFP75', 'RBS1', 21.753803269454018, 172.246196730546],
    'G7': ['B6', 'GFP10', 'RBS2', 29.590661957039284, 164.4093380429607],
    'C4': ['B6', 'GFP10', 'RBS2', 29.590661957039284, 164.4093380429607],
    'C5': ['C6', 'GFP25', 'RBS2', 19.308851308806805, 174.6911486911932],
    'B2': ['C6', 'GFP25', 'RBS2', 19.308851308806805, 174.6911486911932],
    'E8': ['D6', 'GFP50', 'RBS2', 26.603062057401182, 167.3969379425988],
    'E5': ['D6', 'GFP50', 'RBS2', 26.603062057401182, 167.3969379425988],
    'C7': ['E6', 'GFP75', 'RBS2', 21.328422109151145, 172.67157789084885],
    'D10': ['E6', 'GFP75', 'RBS2', 21.328422109151145, 172.67157789084885],
    'E11': ['B7', 'GFP10', 'RBS4', 23.415477366991134, 170.58452263300887],
    'D8': ['B7', 'GFP10', 'RBS4', 23.415477366991134, 170.58452263300887],
    'B6': ['C7', 'GFP25', 'RBS4', 19.277105846322616, 174.72289415367737],
    'E3': ['C7', 'GFP25', 'RBS4', 19.277105846322616, 174.72289415367737],
    'G10': ['D7', 'GFP50', 'RBS4', 21.78411412397232, 172.21588587602767],
    'G3': ['D7', 'GFP50', 'RBS4', 21.78411412397232, 172.21588587602767],
    'F8': ['E7', 'GFP75', 'RBS4', 20.49877750300502, 173.50122249699498],
    'D3': ['E7', 'GFP75', 'RBS4', 20.49877750300502, 173.50122249699498],
    'B5': ['blank', 'blank', 'blank', 0, 194],
    'B8': ['blank', 'blank', 'blank', 0, 194],
    'B10': ['blank', 'blank', 'blank', 0, 194],
    'C11': ['blank', 'blank', 'blank', 0, 194],
    'D11': ['blank', 'blank', 'blank', 0, 194],
    'E4': ['blank', 'blank', 'blank', 0, 194],
    'E6': ['blank', 'blank', 'blank', 0, 194],
    'E7': ['blank', 'bla

Now we add the dictionaries to the base protocol and test to make sure it runs.

In [8]:
# Make a copy of base_protocol.py and name it complete_protocol.py
os.system('cp base_protocol_multichannel.py complete_protocol.py')

# Add the volumes to the complete_protocol.py file
with open('complete_protocol.py', 'a') as file:
    file.write(f'final_positions = {final_positions}\n')
    file.write(f'iptg_volume = {iptg_volume}\n')

# Simulate the protocol
opentrons.simulate.simulate(open(protocol_file))

([{'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'location': H1 of Opentrons 96 Tip Rack 300 µL on 8,
    'text': 'Picking up tip from H1 of Opentrons 96 Tip Rack 300 µL on 8'},
   'logs': [<LogRecord: opentrons.calibration_storage.ot2.tip_length, 30, /home/croots/miniconda3/envs/opentrons/lib/python3.10/site-packages/opentrons/calibration_storage/ot2/tip_length.py, 53, "Tip length calibrations not found for None">]},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 186.11744253414216,
    'location': Location(point=Point(x=278.94, y=133.4, z=7.289999999999999), labware=A1 of USA Scientific 12 Well Reservoir 22 mL on 6),
    'rate': 1.0,
    'text': 'Aspirating 186.11744253414216 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 6 at 150.0 uL/sec'},
   'logs': []},
  {'level': 0,
   'payload': {'instrument': <CustomPipette: p300_single_v1 in LEFT>,
    'volume': 169.1976750310383,
    'locat